## Counting Haplotypes

Some notes on the classic paper

[Excoffier, M. and Slatkin, L. Maximum-Likelihood Estimation of Molecular Haplotype Frequencies in a Diploid Population. Mol. Biol. Evol. 12(5):921-927 (1995)](http://www.uvm.edu/~rsingle/stat380/F04/possible/Excoffier+slatkin-MolBiolEvol-1995.pdf)

Suppose we have two loci, each of which has potential genotype at that locus of  $0,1,2$ representing the number of "reference" alleles.  An individual can have one of nine combined genotypes depending on the situation at each locus.

One can ask a more refined question: what are the underlying haplotypes?  If the individual is, say, of type $22$ then it must be the case that each chromosome is of type $11$.  But if the individual is of type $11$
then there are two possibilities: $(10,01)$ or $(00,11)$.  

This question is of interest because one might want to know if the sites are independent of one another or if there is non-trivial linkage between them.

More generally, if there are $n$ loci in a diploid individual, there are $3^n$ possible combinations of genotypes.  Although it is somewhat of an abuse of language, we follow the language of the paper above and call such a combination a phenotype.  Phenotypes are indexed by vectors of $n$ integers between $0$ and $2$.

We have $2^n$ haplotypes obtained by choosing either the reference or alternate allele at each of the n sites,
so haplotypes are indexed by vectors of $n$ integers between $0$ and $1$.  The phenotype associated with a pair
$\alpha$ and $\beta$ of haplotypes corresponds to the vector sum $\alpha+\beta$.  If a phenotype has a zero or a two at a particular location, then that determines the possible haplotypes at that location.  If the phenotype has a $1$, then at that location there are two possibilities.  As a result, the number of ways that a phenotype
can be decomposed into a sum of haplotypes (taking symmetry into account) is $2^(s-1)$ where $s$ is the number of $1$'s in the phenotype (or $1$ if that number is zero).

### EM algorithm

We're given counts of each of the $3^n$ phenotypes and we want to give the maximum likelihood estimate of the haplotype frequencies under the assumption that there is no linkage between them.  

Let's  consider the case where $n=2$ so that there are $9$ phenotypes and $4$ haplotypes.   The data consists of counts $N_{ij}$ over the nine phenotypes (so $i$ and $j$ are between $0$ and $2$ inclusive). Let $N$ be the sum of the $N_{ij}$. 

We choose initial (prior) estimates for the haplotype frequencies $p_{ij}^{(0)}$ (where here $i$ and $j$ are between $0$ and $1$.)

In light of the phenotype data, we find the expected number of times each haplotype occurs assuming the
haplotype frequency estimates.  In most cases, the phenotype determines the haplotypes, so that
for example phenotype $22$ contributes two copies of haplotype $11$ and phenotype $12$ contributes
$1$ copy of haplotype $01$ and one copy of haplotype $11$.  The only exception is phenotype $11$, which
is made up of either haplotype $00$ and haplotype $11$ or haplotype $10$ and haplotype $01$.  So among
the observed number of individuals with phenotype $11$, how many come from each haplotype?  Given
the estimated haplotype frequencies, we can compute the expected fractions as
$$
E(10,01)=\frac{p_{10}p_{01}}{p_{01}p_{10}+p_{00}p_{11}}
$$
and
$$
E(00,11)=\frac{p_{00}p_{11}}{p_{01}p_{10}+p_{00}p_{11}}.
$$

Putting this information together we can compute the estimated number of times each haplotype appears given the data.  Dividing by the total number of chromosomes, we get the expected frequency of each haplotype assuming
the data and the prior estimates. 

Now we view these frequencies as the maximum likelihood estimate of the true probabilities given the "expected" data.  Now we put those frequencies back through the process to refine them further. 






In [2]:
import numpy as np

In [14]:
p00,p01,p10,p11=.25,.25,.25,.25
a,b=.5,.5

In [15]:

N = np.array([2,1,0,1,a,0,0,0,0,0,0,0,1,b,0,2,1,0,0,1,0,0,b,1,0,0,0,0,0,2,0,a,1,0,1,2]).reshape((4,9))

In [16]:
a=p00*p11/(p00*p11+p10*p01)
b=p10*p01/(p00*p11+p10*p01)

In [25]:
from scipy.stats import binom
from collections import Counter

In [29]:
F=binom(2,.8).rvs(1000)
G=binom(2,.6).rvs(1000)
H=list(zip(F,G))

In [30]:
n=Counter()

In [31]:
for x in H:
    n[x]=n[x]+1

In [32]:
n

Counter({(2, 1): 340,
         (1, 1): 158,
         (1, 0): 51,
         (2, 2): 219,
         (1, 2): 98,
         (2, 0): 97,
         (0, 2): 17,
         (0, 1): 17,
         (0, 0): 3})

In [33]:
V=np.array([3,51,97,17,158,340,17,98,219])

In [36]:
newT=np.dot(N,V)/2000

In [37]:
newT

array([0.0765, 0.114 , 0.235 , 0.5745])